In [28]:
import tkinter as tk
from tkinter import ttk, filedialog
import concurrent.futures
import pandas as pd

In [29]:
global parameters_dict
parameters_dict = {}

def start_gui():
    global parameters_dict

    # Configuration de l'application principale Tkinter
    root = tk.Tk()
    root.title("Explorateur de Fichiers")
    root.geometry("600x500")  # Taille de la fenêtre définie à 600x500

    # Dictionnaire pour stocker les chemins des fichiers sélectionnés

    # Configuration du notebook (les onglets)
    notebook = ttk.Notebook(root)
    notebook.pack(fill='both', expand=True)  # S'assurer que le notebook remplisse l'espace disponible

    # Création des onglets avec configuration pour occuper toute la fenêtre
    frame1 = ttk.Frame(notebook)
    frame2 = ttk.Frame(notebook)
    frame3 = ttk.Frame(notebook)
    frame4 = ttk.Frame(notebook)

    # S'assurer que les cadres remplissent bien l'espace
    for frame in (frame1, frame2, frame3, frame4):
        frame.pack(fill='both', expand=True)  # Utiliser pack avec fill et expand pour les cadres

    notebook.add(frame1, text='INPUT')
    notebook.add(frame2, text='OUTPUT')
    notebook.add(frame3, text='Classyfire')
    notebook.add(frame4, text='NPclassifier')

    # Fonction pour l'explorateur de fichiers
    def browse_file(value_key, filetypes):
        file_path = filedialog.askopenfilename(filetypes=filetypes)
        if file_path:
            parameters_dict[value_key] = file_path  # Enregister le chemin dans le dictionnaire

    # Fonctions de gestion des changements de sélection pour les entrées de texte
    def update_parameters_dict(entry, key):
        parameters_dict[key] = entry.get()

    # Label et bouton de l'explorateur de fichiers pour l'onglet INPUT
    label_input = ttk.Label(frame1, text="Sélectionner un fichier CSV pour INPUT:")
    label_input.pack(pady=10)
    btn_browse_input = ttk.Button(frame1, text="Parcourir",
                                  command=lambda: browse_file('input_file', [('CSV files', '*.csv')]))
    btn_browse_input.pack(pady=10)

    # Label et bouton de l'explorateur de fichiers pour l'onglet OUTPUT
    label_output = ttk.Label(frame2, text="Sélectionner un fichier pour OUTPUT:")
    label_output.pack(pady=10)
    btn_browse_output = ttk.Button(frame2, text="Parcourir",
                                   command=lambda: browse_file('output_file', [('All files', '*.*')]))
    btn_browse_output.pack(pady=10)

    # Gestionnaire pour les onglets "Classyfire"
    label_classyfire = ttk.Label(frame3, text="Activates ?")
    label_classyfire.pack(pady=5)
    # Combobox pour ON/OFF pour Classyfire
    combobox_classyfire_status = ttk.Combobox(frame3, values=["ON", "OFF"])
    combobox_classyfire_status.set("ON")  # Par défaut sur "ON"
    combobox_classyfire_status.pack(pady=2)
    combobox_classyfire_status.bind("<<ComboboxSelected>>",
                                    lambda event: update_parameters_dict(combobox_classyfire_status,
                                                                         'classyfire_status'))

    # Gestionnaire pour les onglets "Classyfire"
    label_classyfire = ttk.Label(frame3, text="inchikeys column:")
    label_classyfire.pack(pady=5)
    # Entry pour saisir directement la colonne inchikeys dans l'onglet Classyfire
    entry_classyfire_column = ttk.Entry(frame3)
    entry_classyfire_column.pack(pady=10)
    entry_classyfire_column.bind("<KeyRelease>",
                                 lambda event: update_parameters_dict(entry_classyfire_column, 'classyfire_column'))

    # Gestionnaire pour les onglets "NPclassifier"
    label_npclassifier = ttk.Label(frame4, text="Activate ?")
    label_npclassifier.pack(pady=5)
    # Combobox pour ON/OFF pour NPclassifier
    combobox_npclassifier_status = ttk.Combobox(frame4, values=["ON", "OFF"])
    combobox_npclassifier_status.set("ON")  # Par défaut sur "ON"
    combobox_npclassifier_status.pack(pady=2)
    combobox_npclassifier_status.bind("<<ComboboxSelected>>",
                                      lambda event: update_parameters_dict(combobox_npclassifier_status,
                                                                           'npclassifier_status'))

    # Gestionnaire pour les onglets "NPclassifier"
    label_npclassifier = ttk.Label(frame4, text="smiles column:")
    label_npclassifier.pack(pady=5)
    # Entry pour saisir directement la colonne smiles dans l'onglet NPclassifier
    entry_npclassifier_column = ttk.Entry(frame4)
    entry_npclassifier_column.pack(pady=10)
    entry_npclassifier_column.bind("<KeyRelease>", lambda event: update_parameters_dict(entry_npclassifier_column,
                                                                                        'npclassifier_column'))

    # Bouton Start qui enregistre tous les paramètres et quitte l'application
    def on_start():
        # Enregistrez les dernières valeurs des entrées de texte
        update_parameters_dict(entry_classyfire_column, 'classyfire_column')
        update_parameters_dict(entry_npclassifier_column, 'npclassifier_column')

        # Enregistrez les valeurs des combobox
        parameters_dict['classyfire_status'] = combobox_classyfire_status.get()
        parameters_dict['npclassifier_status'] = combobox_npclassifier_status.get()

        # Affiche le dictionnaire des paramètres

        # Quitter l'application
        root.destroy()

    start_button = ttk.Button(root, text="Start", command=on_start)
    start_button.pack(pady=20)

    root.mainloop()

In [30]:
# Appel de la fonction pour démarrer l'interface graphique
start_gui()
print(parameters_dict)

{'input_file': 'D:/Stagiaires/Marion/SIDEROPHORE/PubChem_protein_text_siderophore.csv', 'output_file': 'D:/Stagiaires/Marion/SIDEROPHORE/CheBI_Results.xlsx', 'classyfire_column': 'jj', 'npclassifier_column': 'rgr', 'classyfire_status': 'ON', 'npclassifier_status': 'ON'}


In [31]:
global df
df = pd.read_csv(parameters_dict['input_file'], sep=',', quotechar='"', encoding='utf-8')

In [ ]:
print(df.head())

In [ ]:
def interrogation_process(row):
    result = {}

    if parameters_dict['classyfire_status'] == "ON":
        result['classyfire'] = classyfire_interrogation(row)

    if parameters_dict['npclassifier_status'] == "ON":
        result['npclassifier'] = npclassifier_interrogation(row)

    return result

In [ ]:
def run_interrogation(df):
    # Liste pour collecter les résultats
    results = []

    # Fonction pour gérer les threads
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Soumettre les tâches à l'exécuteur
        future_to_row = {executor.submit(interrogation_process, row): row for index, row in df.iterrows()}

        # Collecter les résultats au fur et à mesure qu'ils sont complétés
        for future in concurrent.futures.as_completed(future_to_row):
            result = future.result()
            results.append(result)

    # Retourner les résultats agrégés
    return results